# 07：im2colを用いた効率的な畳み込み処理

---
## 目的
畳み込みやプーリングを効率的に計算するために使用される`im2col`関数について理解する．

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
`time`関数はシステム時刻を取得する関数であり，今回は処理時間の計測に使用します．

In [ ]:
import numpy as np
from time import time

## im2colを用いない畳み込み処理
はじめに愚直に畳み込み処理を計算してみます．

まず，畳み込みを行う際のパラメータを設定します．

次に，設定したパラメータに基づいて，擬似的な画像`image`と畳み込みのカーネル`w`およびバイアス`b`を設定します．
また，画像サイズおよびカーネルサイズ，ストライドから畳み込み処理を適用した場合の特徴マップのサイズ`feature_map_size`を計算し，
畳み込みの結果を格納する配列`feature_map`を生成します．

その後，画像の縦方向と横方向に対するfor文を定義し，1箇所ずつ畳み込み処理を行い，演算結果を`feature_map`の対応する要素へと格納していきます．
この時，for文の前後に`time`関数を適用し時刻を取得することで，処理時間の計算を行います．

In [ ]:
# 畳み込み処理のパラメータの設定（画像サイズ，カーネルサイズ等）
batch_size = 2
img_size = 256
kernel_size = 3
in_channels = 3
out_channels = 7
stride = 1

image = np.arange(batch_size * in_channels * img_size * img_size, dtype=np.float32).reshape(batch_size, in_channels, img_size, img_size) / 100.
w = np.arange(out_channels * in_channels * kernel_size * kernel_size, dtype=np.float32).reshape(out_channels, in_channels, kernel_size, kernel_size) / 100.
b = np.random.randn(out_channels)

feature_map_size = int(img_size - (kernel_size - 1))
feature_map = np.zeros((batch_size, out_channels, feature_map_size, feature_map_size))

conv_start = time()
for y in range(feature_map_size):
    for x in range(feature_map_size):
        for batch in range(batch_size):

            img_patch = image[batch:batch+1, :, y:y+kernel_size, x:x+kernel_size]
            value = np.tensordot(img_patch, w, ((1,2,3), (1,2,3)))
            feature_map[batch, :, y, x] = value + b
conv_end = time()

print(feature_map.shape)
print("processing time (w/ im2col):", conv_end - conv_start, "[s]")

## im2colの実装

上記の愚直な畳み込み処理では，for文を入れ子にして繰り返し演算する必要があるため，pythonでは処理に多くの時間を要します．
より効率的に処理を行うために，`im2col`と呼ばれる配列処理を用いて畳み込み処理が行われます．

`im2col`では，畳み込みの演算を一度に適用することができるように，入力画像の配列を変換する処理を行っています．
具体的には，下の図に示すように，一度の畳み込み演算に該当するデータを取得し，それを横一列に平坦化します．
そして，その配列を縦に並べたものを返します．
![im2col](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/143078/abaf537b-0763-b92a-e714-477497b1f8d4.png)

`im2col`を用いることで，for文で行なっていた演算を一度の行列演算に置き換えることができるため，numpyなどの行列演算ライブラリの高速な演算を活用することができます．

### im2colの定義
上記で説明したim2colを関数として定義します．
まず，関数の引数として，変換対象の配列`input_image`，畳み込みを行う際のカーネルサイズ`kernel_h`, `kernel_w`，パディング，ストライドを与えます．
画像のサイズ，パディング，ストライドから，縦横それぞれ畳み込みを行う回数を`dst_h`, `dst_w`として求めます．

次に，入力画像に対してパディングを行い，`im2col`変換後の値を格納する配列`col`を初期化します．
その後，for文をもちいて，対象となるデータを順次取得し，`col`へと格納していきます．

最後に，`col`の配列を並び替えることで変換が実現されます．

In [ ]:
def im2col(input_image, kernel_h, kernel_w, stride=1, padding=0):
    n, c, h, w = input_image.shape
    
    dst_h = (h + 2 * padding - kernel_h) // stride + 1
    dst_w = (w + 2 * padding - kernel_w) // stride + 1
    
    image = np.pad(input_image, [(0,0), (0,0), (padding, padding), (padding, padding)], 'constant')
    col = np.zeros((n, c, kernel_h, kernel_w, dst_h, dst_w))
    
    for y in range(kernel_h):
        y_max = y + stride * dst_h
        for x in range(kernel_w):
            x_max = x + stride * dst_w
            col[:, :, y, x, :, :] = image[:, :, y:y_max:stride, x:x_max:stride]
    
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(n * dst_h * dst_w, -1)
    return col

上記で定義した`im2col`関数を用いて，入力画像の配列を変換してみます．

In [ ]:
# 畳み込み処理のパラメータの設定（画像サイズ，カーネルサイズ等）
img_size = 7
kernel_size = 3
in_channels = 3
out_channels = 7
stride = 1
padding = 0

x1 = np.random.rand(1, in_channels, img_size, img_size)
col1 = im2col(x1, kernel_size, kernel_size, stride, padding)
print(col1.shape)

x2 = np.random.rand(10, in_channels, img_size, img_size)
col2 = im2col(x2, kernel_size, kernel_size, stride, padding)
print(col2.shape)

## im2colを用いた畳み込み処理
次に上記で定義した`im2col`を用いて，畳み込み処理とプーリング処理を行ってみます．

In [ ]:
# 畳み込み処理のパラメータの設定（画像サイズ，カーネルサイズ等）
batch_size = 2
img_size = 256
kernel_size = 3
in_channels = 3
out_channels = 7
stride = 1
padding = 0

image = np.arange(batch_size * in_channels * img_size * img_size, dtype=np.float32).reshape(batch_size, in_channels, img_size, img_size) / 100.
w = np.arange(out_channels * in_channels * kernel_size * kernel_size, dtype=np.float32).reshape(out_channels, in_channels, kernel_size, kernel_size) / 100.
b = np.random.randn(out_channels)

# forward
out_h = 1 + int((img_size + 2 * padding - kernel_size) / stride)
out_w = 1 + int((img_size + 2 * padding - kernel_size) / stride)

# convolution
conv_start = time()
col = im2col(image, kernel_size, kernel_size, stride, padding)
col_w = w.reshape(out_channels, -1).T

out = np.dot(col, col_w) + b

out = out.reshape(batch_size, out_h, out_w, -1).transpose(0, 3, 1, 2)

conv_end = time()

print(out.shape)
print("processing time (w/ im2col):", conv_end - conv_start, "[s]")

## プーリング処理
`im2col`関数は畳み込みだけでなく，プーリング処理にも用いられます．

まず，for文を用いたプーリング処理を行います．

In [ ]:
pooling_size = 2
stride = 2

image = np.random.randn(2, 3, 256, 256)
n, c, h, w = image.shape

pooled_map = np.zeros((2, 3, int(h / pooling_size), int(w / pooling_size)))

pool_start = time()
for y in range(0, h, pooling_size):
    for x in range(0, h, pooling_size):
        patch = image[:, :, y:y+pooling_size, x:x+pooling_size]
        patch = patch.reshape(-1, pooling_size * pooling_size)
        pool_tmp = np.max(patch, axis=1)
        pool_tmp = pool_tmp.reshape(n, c)
        pooled_map[:, :, int(y/2), int(x/2)] = pool_tmp
pool_end = time()
    
print("processing time (w/o im2col):", pool_end - pool_start, "[s]")

次に，`im2col`を用いたプーリング処理を行なってみます．

In [ ]:
pooling_size = 2
stride = 2

x = np.random.randn(2, 3, 256, 256)
n, c, h, w = x.shape

out_h = int(1 + (h - pooling_size) / stride)
out_w = int(1 + (w - pooling_size) / stride)

pool_start = time()
col = im2col(x, pooling_size, pooling_size, stride, 0)
col = col.reshape(-1, pooling_size * pooling_size)

# arg_max = np.argmax(col, axis=1)
out = np.max(col, axis=1)
out = out.reshape(n, out_h, out_w, c).transpose(0, 3, 1, 2)
pool_end = time()

print("processing time (w/ im2col):", pool_end - pool_start, "[s]")